In [4]:
import pandas as pd
import sklearn 
import scipy
from sklearn import linear_model as lm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold, train_test_split, cross_val_score, StratifiedKFold, LabelKFold, ShuffleSplit
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from mhcflurry.amino_acid import common_amino_acids
from mhcflurry import dataset
from mhcflurry.dataset import Dataset
import matplotlib.pyplot as plt 
% matplotlib inline
import numpy as np
import math 
from mhcflurry import peptide_encoding, amino_acid
import statsmodels.api as sm
from keras import models, layers, optimizers
from keras.optimizers import Adam 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, GlobalAveragePooling1D, Convolution1D, AveragePooling1D, Activation, Flatten
from keras.preprocessing import sequence
from keras.models import Model
from keras.engine import topology
import seaborn as sns
from keras.layers.core import Lambda
from keras import backend as K

Using Theano backend.


In [5]:
ds = Dataset.from_csv("bdata.2009.mhci.public.1.txt")
ds_h = ds.slice(ds.alleles == 'HLA-A0201')

In [6]:
df = pd.read_table("bdata.2009.mhci.public.1.txt")

df['log_meas']=1-np.log(df['meas'])/math.log(50000)
df['peptide_length'] = df['sequence'].str.len()


max_len=df['sequence'].str.len().max()
n_peptides = df['sequence'].count()

def amino_acid_hotshot_encoding(s):
    return common_amino_acids.hotshot_encoding([s],len(s)).flatten().astype(int)
df['hotshot_encoded_peptides'] = df.sequence.apply(lambda seq: amino_acid_hotshot_encoding(seq))

def amino_acid_index_encoding(s, maxlen):
    a = 1+common_amino_acids.index_encoding([s],len(s)).flatten()
    return np.concatenate([a, np.zeros(maxlen-len(a),dtype=int)])
df['index_encoded_peptides'] = df.sequence.apply(lambda seq: amino_acid_index_encoding(seq, max_len))

def new_index_encoding(x):
    y = np.zeros(len(x))
    for i in range(len(x)):
        if x[i] != 0: 
            y[i] = x[i]+i*20
    return y

df['new_index_encoding'] = df.index_encoded_peptides.apply(lambda seq: new_index_encoding(seq))    
def measured_affinity_less_than(Y,k):
    IC50 = 50000**(1-Y)
    return (IC50 < k).astype(int) 

def affinity_label(Y):
    return measured_affinity_less_than(Y,50) + measured_affinity_less_than(Y,500) + measured_affinity_less_than(Y,5000) + measured_affinity_less_than(Y,50000)

df['affinity_label'] = affinity_label(df['log_meas'])
df_h = df[df['mhc']=='HLA-A-0201'][['hotshot_encoded_peptides','index_encoded_peptides','log_meas','peptide_length','new_index_encoding']]
X = np.array(list(df_h['index_encoded_peptides']))
X_new = np.array(list(df_h['new_index_encoding']))
y = np.array(list(df_h['log_meas']))
y[y<0]=0

def first_and_last_three(Y):
    k = np.count_nonzero(Y)
    return np.concatenate([Y[:3],Y[-3+k:k]])
def first_and_last_four(Y):
    k = np.count_nonzero(Y)
    return np.concatenate([Y[:4],Y[-4+k:k]])
def first_and_last_two(Y):
    k = np.count_nonzero(Y)
    return np.concatenate([Y[:2],Y[-2+k:k]])
def cut_the_zeros(Y):
    k = np.count_nonzero(Y)
    return Y[:k]

X_44 = np.apply_along_axis(first_and_last_four,1,X)
X_33 = np.apply_along_axis(first_and_last_three,1,X)
X_22 = np.apply_along_axis(first_and_last_two,1,X)

nine_mers = np.array(list(df_h[df_h['peptide_length']==9]['index_encoded_peptides']))
nine_mers_new = np.array(list(df_h[df_h['peptide_length']==9]['new_index_encoding']))
X_9 = np.apply_along_axis(cut_the_zeros,1,nine_mers)
X_9_new = np.apply_along_axis(cut_the_zeros,1,nine_mers_new)
y_9 = np.array(list(df_h[df_h['peptide_length']==9]['log_meas']))

In [7]:
def regroup_together(affinities, weights , original_indices):
    affinities = affinities.ravel()
    weights = weights.ravel()
    
    assert affinities.shape == weights.shape, "%s should be %s" % (affinities.shape, weights.shape)
    assert affinities.shape == original_indices.shape
    assert len(affinities) == len(affinities.ravel())
    
    weighted_affinities = (affinities * weights)
    index_set = set(original_indices)
    n_indices = len(index_set)
    result_order = {original_index: i for (i, original_index) in enumerate(sorted(index_set))}
    result = np.zeros(n_indices)
    for i, x in enumerate(weighted_affinities):
        result_idx = result_order[original_indices[i]]
        result[result_idx] += x
    return result

def slicing(dataset, index, i):
    return dataset.slice(index).kmer_index_encoding()[i]

def label_transform(array):
    result = 1-np.log(array)/math.log(50000)
    result[result<0]=0
    return result

def index_to_hotshot_encoding(index_encoded_nine_mer):
    result = np.zeros((9,21))
    for position, amino_acid in enumerate(index_encoded_nine_mer):
        result[position][amino_acid]= 1
    return result.flatten()

def real_labels(dataset,index):
    
    y = label_transform(slicing(dataset,index,1))
    weights = slicing(dataset,index,2)
    original_indices = slicing(dataset,index,3)
    
    return regroup_together(y, weights , original_indices)

def fit(model,dataset,index, neural_network = False, hotshot = False): # to be left out or modified 
    
    X = slicing(dataset,index,0)
    
    if (hotshot == True):
        X = np.apply_along_axis(index_to_hotshot_encoding, 1, X)
        
    y = label_transform(slicing(dataset,index,1))
    weights = slicing(dataset,index,2)
    
    if (neural_network == True):
        model.fit(X, y, sample_weight = weights, batch_size = 16, nb_epoch = 1)
    else: 
        model.fit(X, y, sample_weight = weights)
        
        
def fit_new(model,dataset,index, neural_network = False):
    X = slicing(dataset,index,0)
    y = label_transform(slicing(dataset,index,1))
    weights = slicing(dataset,index,2)
    X_new = np.apply_along_axis(new_index_encoding, 1, X) 
    
    if (neural_network == True):
        model.fit(X_new, y, sample_weight = weights, batch_size = 16, nb_epoch = 1)
    else: 
        model.fit(X_new, y, sample_weight = weights)
        
        
def predict(model, dataset, index, hotshot = False):
    
    X = slicing(dataset,index,0)
    
    if (hotshot == True):
        X = np.apply_along_axis(index_to_hotshot_encoding, 1, X)
        
    weights = slicing(dataset,index,2)
    original_indices = slicing(dataset,index,3)
    
    return regroup_together(model.predict(X), weights , original_indices)

def AUC(model, dataset, index, hotshot = False):
        
    real_affinity = measured_affinity_less_than(real_labels(dataset,index),500)
    predicted_affinity = predict(model, dataset, index, hotshot = hotshot)
    
    return roc_auc_score(real_affinity, predicted_affinity)

def AUC_simple(model, features, labels, index):
    real_affinity = measured_affinity_less_than(labels[index],500)
    predicted_affinity = model.predict(features[index])
    
    return roc_auc_score(real_affinity, predicted_affinity)

def AUC_simple_average(model_1, model_2, features, labels, index):
    real_affinity = measured_affinity_less_than(labels[index],500)
    predicted_affinity_1 = model_1.predict(features[index])
    predicted_affinity_2 = model_2.predict(features[index])
    predicted_affinity = 0.5* predicted_affinity_1 + 0.5* predicted_affinity_2
    
    return roc_auc_score(real_affinity, predicted_affinity)
def split_by_length(X,index,length=9):
    length_idx = np.array([i for i in index if (np.count_nonzero(X[i])==length)])
    non_length_idx = np.array([i for i in index if (np.count_nonzero(X[i])!=length)])
    return index, length_idx, non_length_idx
from numpy import ma 

array_test = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
position = [[1,0,0,0], [0,1,0,0], [0,0,1,0]]
values = 0
mx = ma.masked_array(array_test, mask=position, fill_value =0)
print(mx)
def sum_character_deletion(model,array,length): 
    result = np.zeros(len(array))
    for i in range(length):
        position_matrix = np.zeros((len(array),length))
        position_matrix[:,i] = 1 
        mx = ma.masked_array(array, mask=position_matrix)
        result = result + model.predict(mx)
        
    return result / length
        
def random_dropout_prediction_by_lentgh(model,array,length):
    array_of_lengths = np.apply_along_axis(np.count_nonzero,1,array)
    #print("array shape", array.shape, "array of lengths shape", array_of_lengths.shape)
    bool_array = (array_of_lengths == length)
    #print("bool_array shape", bool_array.shape)
    result = np.zeros(len(array))
    #print("result shape", result.shape)
    for i in range(length):
        position_matrix = np.zeros(array.shape)
        position_matrix[:,i] = 1 
        print("array shape", array.shape, "position_matrix shape", position_matrix.shape)
        mx = ma.masked_array(array, mask=position_matrix)
        result[bool_array] = result[bool_array] + model.predict(mx)
    
    #print("result[bool_array] shape", result[bool_array].shape, "model prediction shape", model.predict(array[bool_array]).shape)
    return result/length

def random_dropout_array_prediction(model,array):
    array_of_lengths = np.apply_along_axis(np.count_nonzero,1,array)
    result = np.zeros(len(array))
    for length in np.unique(array_of_lengths):
        result = result + random_dropout_prediction_by_lentgh(model,array,length)
    return result

def AUC_random_dropout(model, features, labels, index):
    real_affinity = measured_affinity_less_than(labels[index],500)
    predicted_affinity = random_dropout_array_prediction(model, features[index])
    
    return roc_auc_score(real_affinity, predicted_affinity)

[[-- 2 3 4]
 [5 -- 7 8]
 [9 10 -- 12]]


# Training on 9 mers 

In [ ]:
folds = 3
batch_size_nn = 16
batch_size_lstm = 16
hidden = 50
dropout_probability = 0.25

n_epochs = 50
epoch = 0



nn_8_aucs = np.zeros((2, folds,n_epochs))
nn_6_aucs = np.zeros((2, folds,n_epochs))
nn_4_aucs = np.zeros((2, folds,n_epochs))
nn_2_aucs = np.zeros((2, folds,n_epochs))
nn_aucs = np.zeros((2, folds,n_epochs))


for i, (train_idx, test_idx) in enumerate(KFold(len(X_9),folds, shuffle=True)):
    
    list_index = train_idx, test_idx

    # nn_8    
    sequence = Input( shape= (9, ),dtype='int32')
    embedded = Embedding(input_dim = 21, input_length = 9, output_dim= 32)(sequence)
    
    z_0 = GlobalAveragePooling1D()(embedded)
    z_1 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 2, border_mode='valid', activation='sigmoid')(embedded))
    z_2 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 3, border_mode='valid', activation='sigmoid')(embedded))
    z_3 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 4, border_mode='valid', activation='sigmoid')(embedded))
    z_4 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 5, border_mode='valid', activation='sigmoid')(embedded))
    z_5 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 6, border_mode='valid', activation='sigmoid')(embedded))
    z_6 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 7, border_mode='valid', activation='sigmoid')(embedded))
    z_7 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 8, border_mode='valid', activation='sigmoid')(embedded))
    z_8 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 9, border_mode='valid', activation='sigmoid')(embedded))

    z = merge([z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,z_8], mode = 'concat', concat_axis=-1)
    hidden_layer = Dense(10, init='glorot_uniform', activation = 'sigmoid')(z)
    
    
    after_dp = Dropout(dropout_probability)(hidden_layer)
    output = Dense(1, init='glorot_uniform', activation = 'sigmoid')(after_dp)
    nn_8 = Model(input = sequence, output = output)
    nn_8.compile(optimizer = 'adam', loss='mean_squared_error')

    #nn_6
    sequence = Input( shape= (9, ),dtype='int32')
    embedded = Embedding(input_dim = 21, input_length = 9, output_dim= 32)(sequence)
    
    z_0 = GlobalAveragePooling1D()(embedded)
    z_1 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 2, border_mode='valid', activation='sigmoid')(embedded))
    z_2 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 3, border_mode='valid', activation='sigmoid')(embedded))
    z_3 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 4, border_mode='valid', activation='sigmoid')(embedded))
    z_4 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 5, border_mode='valid', activation='sigmoid')(embedded))
    z_5 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 6, border_mode='valid', activation='sigmoid')(embedded))
    z_6 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 7, border_mode='valid', activation='sigmoid')(embedded))
    
    z = merge([z_0,z_1,z_2,z_3,z_4,z_5,z_6], mode = 'concat', concat_axis=-1)
    hidden_layer = Dense(10, init='glorot_uniform', activation = 'sigmoid')(z)
    
    after_dp = Dropout(dropout_probability)(hidden_layer)
    output = Dense(1, init='glorot_uniform', activation = 'sigmoid')(after_dp)
    nn_6 = Model(input = sequence, output = output)
    nn_6.compile(optimizer = 'adam', loss='mean_squared_error')
    
    # nn_4
    sequence = Input( shape= (9, ),dtype='int32')
    embedded = Embedding(input_dim = 21, input_length = 9, output_dim= 32)(sequence)
    
    z_0 = GlobalAveragePooling1D()(embedded)
    z_1 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 2, border_mode='valid', activation='sigmoid')(embedded))
    z_2 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 3, border_mode='valid', activation='sigmoid')(embedded))
    z_3 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 4, border_mode='valid', activation='sigmoid')(embedded))
    z_4 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 5, border_mode='valid', activation='sigmoid')(embedded))
    
    z = merge([z_0,z_1,z_2,z_3,z_4], mode = 'concat', concat_axis=-1)
    hidden_layer = Dense(10, init='glorot_uniform', activation = 'sigmoid')(z)
    
    after_dp = Dropout(dropout_probability)(hidden_layer)
    output = Dense(1, init='glorot_uniform', activation = 'sigmoid')(after_dp)
    nn_4 = Model(input = sequence, output = output)
    nn_4.compile(optimizer = 'adam', loss='mean_squared_error')
    
    # nn_2
    sequence = Input( shape= (9, ),dtype='int32')
    embedded = Embedding(input_dim = 21, input_length = 9, output_dim= 32)(sequence)
    
    z_0 = GlobalAveragePooling1D()(embedded)
    z_1 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 2, border_mode='valid', activation='sigmoid')(embedded))
    z_2 = GlobalAveragePooling1D()(Convolution1D(nb_filter = 32, input_shape = (9, 32), filter_length = 3, border_mode='valid', activation='sigmoid')(embedded))
    
    z = merge([z_0,z_1,z_2], mode = 'concat', concat_axis=-1)
    hidden_layer = Dense(10, init='glorot_uniform', activation = 'sigmoid')(z)
    
    after_dp = Dropout(dropout_probability)(hidden_layer)
    output = Dense(1, init='glorot_uniform', activation = 'sigmoid')(after_dp)
    nn_2 = Model(input = sequence, output = output)
    nn_2.compile(optimizer = 'adam', loss='mean_squared_error')
    
    
    #nn
    sequence = Input( shape= (9, ),dtype='int32')
    embedded = Embedding(input_dim = 21, input_length = 9, output_dim= 32)(sequence)
    
    z = Flatten()(embedded)
    
    hidden_layer = Dense(10, init='glorot_uniform', activation = 'sigmoid')(z)
    
    after_dp = Dropout(dropout_probability)(hidden_layer)
    output = Dense(1, init='glorot_uniform', activation = 'sigmoid')(after_dp)
    nn = Model(input = sequence, output = output)
    nn.compile(optimizer = 'adam', loss='mean_squared_error')
    
    for epoch in range(n_epochs):
        
        
        #nn
        

            
            
        #deep_nn
        nn_8.fit(X_9[train_idx],y_9[train_idx], batch_size = 16, nb_epoch = 1)
        for k, index in enumerate([train_idx,test_idx]):
            nn_8_aucs[k][i][epoch] = AUC_simple(nn_8, X_9, y_9, index)
            
            
        nn_6.fit(X_9[train_idx],y_9[train_idx], batch_size = 16, nb_epoch = 1)
        for k, index in enumerate([train_idx,test_idx]):
            nn_6_aucs[k][i][epoch] = AUC_simple(nn_6, X_9, y_9, index)
            
        nn_4.fit(X_9[train_idx],y_9[train_idx], batch_size = 16, nb_epoch = 1)
        for k, index in enumerate([train_idx,test_idx]):
            nn_4_aucs[k][i][epoch] = AUC_simple(nn_4, X_9, y_9, index)
        
        nn_2.fit(X_9[train_idx],y_9[train_idx], batch_size = 16, nb_epoch = 1)
        for k, index in enumerate([train_idx,test_idx]):
            nn_2_aucs[k][i][epoch] = AUC_simple(nn_2, X_9, y_9, index)
            
        nn.fit(X_9[train_idx],y_9[train_idx], batch_size = 16, nb_epoch = 1)
        for k, index in enumerate([train_idx,test_idx]):
            nn_aucs[k][i][epoch] = AUC_simple(nn, X_9, y_9, index)
 
    
        
        print("test AUC :",  nn_aucs[1][i][epoch], nn_8_aucs[1][i][epoch], nn_6_aucs[1][i][epoch], nn_4_aucs[1][i][epoch], nn_2_aucs[1][i][epoch] , i, epoch)  
      





In [ ]:
plt.plot( np.arange(0,n_epochs,1), nn_aucs[1,:,:].mean(axis=0),color='r',marker='*', linestyle='-', label ="test NN ")
plt.plot( np.arange(0,n_epochs,1), nn_8_aucs[1,:,:].mean(axis=0),color='b',marker='*', linestyle='-', label ="test NN_8 ")
plt.plot( np.arange(0,n_epochs,1), nn_6_aucs[1,:,:].mean(axis=0),color='navy',marker='*', linestyle='-', label ="test NN_6 ")
plt.plot( np.arange(0,n_epochs,1), nn_4_aucs[1,:,:].mean(axis=0),color='green',marker='*', linestyle='-', label ="test NN_4 ")
plt.plot( np.arange(0,n_epochs,1), nn_2_aucs[1,:,:].mean(axis=0),color='orange',marker='*', linestyle='-', label ="test NN_2 ")
plt.legend(loc=4)